## Address validation build

In [1]:
import pandas as pd

target_df = pd.read_csv('/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_list.csv')
target_df.head()

FileNotFoundError: [Errno 2] File /Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_list.csv does not exist: '/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_list.csv'

In [ ]:
import os, re
from dotenv import load_dotenv
from smartystreets_python_sdk import StaticCredentials, exceptions, Batch, ClientBuilder
from smartystreets_python_sdk.us_street import Lookup as StreetLookup

# pass into fun the while df - other columns are needed

address_df = target_df.sample(n=10)

load_dotenv()
auth_id = os.environ['SMARTY_AUTH_ID']
auth_token = os.environ['SMARTY_AUTH_TOKEN']

credentials = StaticCredentials(auth_id, auth_token)

client = ClientBuilder(credentials).build_us_street_api_client()

## batch validation
# batch = Batch()

# # Documentation for input fields can be found at:
# # https://smartystreets.com/docs/us-street-api#input-fields

# for address in enumerate(address_list): # key of the dict are indicies form target_df
# batch.add(StreetLookup())
# #  batch[i].business = address_list[index]['restaurant_name']
# batch[i].street = address_list[key]['street']
# batch[i].city = address_list[key]['city']
# batch[i].state = address_list[key]['state']
# batch[i].match = "strict"

def single_validate(address):
    lookup = StreetLookup()
    lookup.street = address
    lookup.match = "strict"
    try:
        client.send_lookup(lookup)
    except exceptions.SmartyException as err:
        print(err)
        return 100
    
    result = lookup.result
    if not result:
        print("\nInvalid address. Added to log file")
        print(address)
        with open("invalid_addresses.txt", "a+") as myfile:
            myfile.write("{}\n".format(address))
        return
    else:
        return result
        
# single validation with response check and modified attempt

for address in address_df['address']:
    address = address.lower()
    res = single_validate(address)
    if not res: # modify address and try again
        address = ' '.join([re.sub('^w$', 'west', x) for x in address.split()])
    if not res and address == '130-09 Hillside Avenue, Elmhurst, NY':
        addres = '167-09 Hillside Avenue, Jamaica, NY 11432'
        res = single_validate(address)
    if not res:
        text = address.split(',')
        address = ','.join([text[0], 'queens', text[2]])
        res = single_validate(address)
    
    if not res:
            continue
    # if address was found
    candidate = res[0]
    print("\nAddress [{}] is valid.\nThere's {} candidates and the first is:". format(address, len(res)))
    print('Street address is: ' + candidate.delivery_line_1)
    print('                   ' + candidate.last_line)
    print("City: " + candidate.components.city_name)
    print("State: " + candidate.components.state_abbreviation)
    print("zipcode: {}".format(candidate.components.zipcode))


In [ ]:
target_df = pd.read_csv('/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_list.csv')
target_df.head()

# Target fix

In [ ]:
from database import Database

In [ ]:
db = Database()
neighborhood_df = db.select_df('''SELECT * FROM coordinates''')

In [ ]:
from Google_business_search import get_name_from_coordinates
import json 

neighborhoods = neighborhood_df.copy()
neighborhoods['reversed_geocode'] = None
c=1
for i, row in neighborhoods.iterrows():
    
    neighborhoods.loc[i, 'reversed_geocode'] = get_name_from_coordinates(row.lat, row.lng)
    if neighborhoods.loc[i, 'reversed_geocode'] is None: print('Reverse Geocode failed for index:{}'.format(i))
    print('[{0}/{1}]'.format(c, neighborhoods.shape[0]), end='\r', flush=True)
    c+=1

sum(neighborhoods['reversed_geocode'].isnull())
# neighborhoods['reversed_name'] = neighborhoods.apply(lambda row: get_name_from_coordinates(row.lat, row.lng), axis=1)
# sum(neighborhoods['reversed_name'].isnull())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ny_or_not = neighborhoods.dropna(subset=['reversed_geocode']).apply(lambda x: 'NY' in x['reversed_geocode'], axis=1)
null_count = sum(neighborhoods['reversed_geocode'].isnull())
plot_count = [sum(ny_or_not), len(ny_or_not)-sum(ny_or_not), null_count]
plt.figure(figsize=(14, 8))
g = sns.barplot(x=['NY','Other', 'None'], y=plot_count)
g_labels = g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')

# Get Target variable from sources:
## 1. Halal Foodies

In [ ]:
from pykml import parser
import os

from importlib import reload
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

kml_file = '/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_feature/MuslimFoodies_Halal_Map_NYC.kml'

with open(kml_file) as f:
    doc = parser.parse(f).getroot()



df = pd.DataFrame(columns=['name', 'coordinates', 'category', 'url'])

# verified halal listings
start_rows = df.shape[0]
for e in doc.Document.Folder.Placemark:
    row = {'name' : names.append(e.name.text),
            'coordinates' : e.Point.coordinates.text,
            'category' : category.append('MFoodies'),
            'url' : None}
    df = df.append(row, ignore_index=True)
print('found {} restaurants in MFoodies list'.format(df.shape[0]-start_rows))

# partially halal listings
start_rows = df.shape[0]
for e in doc.Document.Folder[1].Placemark:
    row = {'name' : names.append(e.name.text),
            'coordinates' : coords.append(e.Point.coordinates.text),
            'category' : category.append('MFoodies_partial'),
            'url' : None}
    df = df.append(row, ignore_index=True)
print('found {} restaurants in MFoodies partial list'.format(df.shape[0]-start_rows))

df.to_csv('/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/data/data_collection/target_feature/MFoodies_list.csv', index=False)


In [ ]:
# e.Point.coordinates.text.split()[0].split(',')[:2]
''.join(e.Point.coordinates.text.split()[0].split(',')[:2])

## 2. Get list with coordinates from Zabiha.
might still be useful to match by name & coordinates and see if there's results from Google/yelp


In [ ]:
import requests
from bs4 import BeautifulSoup


zabiha_bk_url = 'https://www.zabihah.com/sub/United-States/New-York/New-York-City/Manhattan/NEwhtS6OzN'
html_text = requests.get(zabiha_bk_url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [ ]:
import re

tracks = soup.find_all('script')

# count = 0
# for track in tracks:
#     print(count, track)
#     count += 1
map_data = tracks[17]


In [ ]:
# append dictionary to dataframe then write to csv
for line in map_data.string.split('\n'):
    if 'coordinate' in line and 'title' in line:
        coord = re.search(r'(?<=Coordinate\()(.*)(?=\), )', line).group(0)
        coord = coord.replace(' ', '')
        title = re.search(r'(?<=title: \")(.*)(?=\", )', line).group(0)
        url = re.search(r'(?<=url: \")(.*)(?=\" })', line).group(0)
        row = {'name' : title,
                'coordinates' : coord,
                'borough' : 'Manhattan',
                'url' : 'https://www.zabihah.com' + url}

## 3. Extract list from pesonal Yelp collections
might require scraping

In [ ]:
'''Scraping restaurant list from my yelp collection'''

from selenium import webdriver as Webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from database import Database
import os
from dotenv import load_dotenv
import time, random
from review_scraper import _get_webdriver, _close_webdriver

collection_url = 'https://www.yelp.com/collection/UciUoQ1AvCrJ8Hoo3rHZGg'

'''Scraping restaurant list from my yelp collection'''

from selenium import webdriver as Webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import os, sys
from dotenv import load_dotenv
import time, random

# add personal scripts to sys.path
modules_path = [os.path.abspath(os.path.join('.')+'/src/data/data_collection')] #, os.path.abspath(os.path.join('.')+'/web/pages')
for module in modules_path:
    if module not in sys.path:
        sys.path.append(module)
from database import Database
from review_scraper import _get_webdriver, _close_webdriver

def scrape_yelp_collection(collection_url):
    webdriver = _get_webdriver()
    webdriver.get(collection_url)

    scrollable_pane_css = 'div.collection-content.collection-left-pane.u-inline-block'
    scrollable = WebDriverWait(webdriver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, scrollable_pane_css)))
    _yelp_infinite_scroll(scrollable) # scroll to reveal all results
    rest_list = webdriver.find_elements_by_xpath('//a[@data-analytics-label="biz-name"]')
    for rest in rest_list:
        name = rest.find_element_by_xpath('/./span').text
        addr_city = rest.find_element_by_xpath('/../../small/span[@class="addr-city"]').text
        print(name, addr_city)
    _close_webdriver(webdriver)

def _yelp_infinite_scroll(element):
    element._parent.execute_script("""
        var element = arguments[0];
        function scroll(element) {
        element.scrollTo(0, element.scrollHeight);
        }

        var oldHeight = element.scrollHeight;
        for(var i = 0; i<100; i++) {
            setTimeout(scroll(element), 5000);
            await new Promise(resolve => setTimeout(resolve, 2000));
            if (element.scrollHeight > oldHeight) {
                oldHeight = element.scrollHeight;
            } else {
                i = 100;
            }
        }
        """, element)

def main():
    my_collection_url = 'https://www.yelp.com/collection/UciUoQ1AvCrJ8Hoo3rHZGg'
    scrape_yelp_collection(my_collection_url)

main()

    
    
    

# Troubleshoot neighborhood list and search coverage

In [ ]:
from database import Database

db = Database()
neighborhood_df = db.select_df('''SELECT * FROM coordinates''')

In [ ]:
neighborhood_df.head()

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(neighborhood_df, lat="lat", lon="lng", hover_name="neighborhood", hover_data=["neighborhood", "id"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### get a new neighborhood list from Wikipedia

In [ ]:
from review_scraper import _get_webdriver, _close_webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import os, sys
modules_path = [os.path.abspath(os.path.join('.')+'/src/data/data_collection')]
for module in modules_path:
    if module not in sys.path:
        sys.path.append(module)
from Google_business_search import get_corrdinates_from_name
from database import Database
import pandas as pd
# from database import Database

url = WIKI_URL = 'https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City'
nbr_df = pd.DataFrame(columns=['neighborhood', 'lat', 'lng'])
# name_list = []
# coords = []
# def scrape_wiki_list(url):
try:
#     # convert neighborhood_list to neighborhood_coordinates and update database
#     db = Database()
#     update_sql = """INSERT INTO coordinates (neighborhood, lat, lng)
#                     VALUES (%s, %s, %s)"""
    webdriver = _get_webdriver()
    webdriver.get(url)
    table_xpath = '//table[@class="wikitable sortable jquery-tablesorter"]'
    WebDriverWait(webdriver, 10).until(EC.presence_of_element_located((By.XPATH, table_xpath)))
    rows = webdriver.find_elements_by_xpath(table_xpath + '/tbody/tr')
    c=0
    for row in rows:
        city = row.find_element_by_xpath('./td[1]/a').text
        names = row.find_elements_by_xpath('./td[5]/a')
        for name in names:
            name_string = ''.join(name.get_attribute('title').split())
#             name_list.append(name_string)
            lat, lng = get_corrdinates_from_name(name_string)
#             coords.append([lat, lng])
            new_row = { 'neighborhood' : name_string, 'lat' : lat, 'lng' : lng}
            nbr_df = nbr_df.append(new_row, ignore_index=True)
#             json_obj = get_corrdinates_from_name(name_string)
#             print(json_obj)
        c+=1
        print('Progress: [{}/{}]'.format(c, len(rows)), end='\r', flush=True)
#             db.insert_row(update_sql, *(neighborhood, lat, lng))

finally:
    _close_webdriver(webdriver)

nbr_df.head()

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(nbr_df, lat="lat", lon="lng", hover_name="neighborhood", hover_data=["neighborhood"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Random testing 

In [1]:
from review_scraper import scrape_google_reviews

google_test_url = 'https://maps.google.com/?cid=4988939335835406517'
google_id = 'randomID8083rpoeu'
data = scrape_google_reviews(google_url=google_test_url, google_id=google_id)

attempting to find search input
attempting to cick search button
Scrolling to show all reviews
Scraped 12 reviews from google business id #randomID8083rpoeu


In [2]:
data

[['Small spaces, reasonable prices, ordered chichen salad, chicken kabab, chicken baryani, chicken gyro, and naan. Everything was okay except baryani which, was not to our baryani standard. Neither i would call it terrible. Halal sign is a question mark because of sikh management. Its not a muslim restaurant.',
  '10 months ago',
  ' 3 stars '],
 ['I have been ordering food from here for years now and every time the food is fresh tasty and delicious. The service is always delightful. I moved to Brooklyn and I still come back here for great indian halal food. Best place ever, i tried other places and nothing compares to Punjabi Kabab and Gyro.',
  '10 months ago',
  ' 5 stars '],
 ['One of the best gyro experience. Think about having it from a restaurant instead of a cart. And a great option of different salads to choose from. The owner is there at the restaurant most of the time who is an amazing host. Must have Gyro in queens which beats Halal Guys.',
  '10 months ago',
  ' 5 stars ']

# Google image url form image reference

In [21]:
import requests, json, time
#import database_func as db
from database import Database
from dotenv import load_dotenv
import os

load_dotenv()
API_key = os.getenv('GOOGLE_API_KEY_3')
image_ref = 'CmRaAAAAISFXZ2RAvYH6kgZOCixBoPiLsAEEpw-Lz1SN_raOkZoBc9VKHZye8kj_PpYWbLRJ5IwLBjMNgfSpi5fftmRdb-g1cQjd3c9yMD_M-e5fAz2JASDzo0mNr6Y9fhBg9ZGzEhAzQiN5INe023qvmnVthb63GhRwqZdwZk9y6t8nkqpeWfWn8frS_w'

URL = ('https://maps.googleapis.com/maps/api/place/photo?maxwidth=400&photoreference='
     + image_ref + '&key='+ API_key)
response = requests.get(URL)
# json_obj = json.loads(response.text)
# print(json_obj)

In [22]:
response.status_code

200

# Coordinates formating

In [15]:
### sample location
import sys, os
sys.path.append( os.getcwd() + '/src/data/data_collection/')

from database import Database

db = Database()
# fetch the list of NYC neighborhoods' coordinates
get_lat_lng = '''SELECT neighborhood, lat, lng
                    FROM coordinates'''
results = db.select_rows(get_lat_lng)
coordinates_list = [coord for coord in results]
coordinates_list = [[coord[0].replace('+', ' '), ','.join([str(coord[1]),str(coord[2])])] for coord in results]
coordinates_list

[['Bath Beach,Brooklyn', '40.6038852,-74.0062078'],
 ['Allerton,Bronx', '40.8631189,-73.8616476'],
 ['Battery Park City,Manhattan', '40.7032775,-74.0170279'],
 ['Arverne,Queens', '40.592658,-73.7977928'],
 ['Annadale,Staten Island', '40.5445496,-74.1765324'],
 ['Bay Ridge,Brooklyn', '40.6263732,-74.0298767'],
 ['Bathgate,Bronx', '55.9024,-3.643118'],
 ['Beekman Place,Manhattan', '40.7533089,-73.9647957'],
 ['Astoria,Queens', '46.1878841,-123.8312534'],
 ['Arden Heights,Staten Island', '40.556413,-74.1735044'],
 ['Bedford Stuyvesant,Brooklyn', '40.6872176,-73.9417735'],
 ['Baychester,Bronx', '40.8693862,-73.83308439999999'],
 ['Carnegie Hill,Manhattan', '40.7844653,-73.9550857'],
 ['Astoria Heights,Queens', '40.7687747,-73.8915777'],
 ['Arlington,Staten Island', '40.7763877,-73.0485198'],
 ['Bensonhurst,Brooklyn', '40.6139307,-73.9921833'],
 ['Bedford Park,Bronx', '41.7641723,-87.7812524'],
 ['Chelsea,Manhattan', '40.7465004,-74.00137370000002'],
 ['Auburndale,Queens', '28.0652975,-81.7

In [28]:
import requests, json

API_key = '**************************'

def _get_google_business_details(place_id, fields='url,price_level,rating,user_ratings_total'):
    URL = ('https://maps.googleapis.com/maps/api/place/details/json?placeid='
    + place_id + '&fields=' + fields + '&key='+API_key)
    r = requests.get(URL)
    response = r.text
    json_obj = json.loads(response)
    place_details = json_obj["result"]
    res_dict = dict()
    for var in fields.split(','):
        try:
            res_dict[var] = place_details[var]
        except:
            res_dict[var] = 0
    return res_dict

url2,rating2 = _get_google_business_details(place_id='ChIJCQ5RQ-8JyIkR5xJvwrcQ1Qk', fields='url,rating').values()